<a href="https://colab.research.google.com/github/shwetapriya64/16S-amp-seq-classification/blob/k-mer_embedding/generate_kmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Generate k-mer file of 16S DNA sequences **

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/Datasets/Bac_16s_data/

In [ ]:
##import libraries
import gzip
import shutil
import pandas as pd
import numpy as np
from Bio import SeqIO
import sys
import re
import os
import io
import math
from itertools import product
import six.moves.cPickle
import random
import gensim
#from gensim.models import Word2Vec
#from gensim.models.word2vec import LineSentence


In [ ]:
#install biopython to read fasta/fastq files
pip install biopython

In [ ]:

#import training file to be used to create k-mer embedding-matrix and train word2vec model
input_file = 'trainset18_062020.fa' ##training set from RDP classifier website

sequences = []
seq_ids = []
Classification = []
seq_dict = {}
with open(input_file) as f:
    for record in SeqIO.parse(f, 'fasta'):
      Seq_ID = record.id
      line = record.description
      rank = re.split(r'\t+', line)[1]
      classes = re.split(r';+', rank)[1:]
      seq_ids.append(Seq_ID)
      Classification.append(classes)
      seq_dict[Seq_ID] = record.seq
      sequences.append(record.seq)
      seq_ids.append(record.id)

In [ ]:
##create taxonomy file
df = pd.DataFrame()
df['Seq_ID'] = Seq_ids
df['Classification'] = Classification
df_expanded = df['Classification'].apply(pd.Series)
df = df.drop(['Classification'], axis = 1)
df_taxonomy = pd.concat([df, df_expanded], axis = 1)

In [ ]:
##data cleaning and resizing
df_taxonomy.drop([1, 6, 7], axis = 1, inplace=True)
df_taxonomy.columns = ['Seq_ID', 'Phylum', 'Class', 'Order', 'Family', 'Genus']
df = df_taxonomy[df_taxonomy['Phylum'].isin(['Actinobacteria','Proteobacteria','Acidobacteria', 'Firmicutes', 'Tenericutes', 'Cyanobacteria/Chloroplast', 'Bacteroidetes', 'Verrucomicrobia', 'Chloroflexi','Nitrospirae', 'Gemmatimonadetes', 'Planctomycetes'])]
df.to_csv('/content/gdrive/MyDrive/Datasets/Bac_16s_data/train_tax.csv', index = None)

In [ ]:
#import taxonomy file
df1 = pd.read_csv('/content/gdrive/MyDrive/Datasets/Bac_16s_data/train_tax.csv')

In [ ]:
# create dataframe of only the sequences to be converted to k-mers embedding matrix
seq_list = []
for i in range(len(df1)):
  sequence = seq_dict[df1.Seq_ID[i]]
  seq_list.append((str(sequence)).upper())
seq_df = pd.DataFrame(seq_list)
seq_df.columns = ['Sequences']
seq_df.head()

,Sequences
0,GAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGGAAAGGTC...
1,GTTTGATCCTGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACAT...
2,TAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAAC...
3,AAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCATGCTTAAC...
4,ATTGAACGCTGGCGGCATGCCTTACACATGCAAGTCGAACGGTAAC...


In [ ]:
#generate k-mers from the sequence file of each sequnence 
def getKmers(sequence, size=12):
    return [sequence[x:x+size] for x in range(len(sequence) - size + 1)]

seq_df['Kmers'] = seq_df.apply(lambda x: getKmers(x['Sequences']), axis = 1)
seq_df.drop('Sequences', axis=1, inplace = True)
seq_df.head()
#seq_df.to_csv(r'/content/gdrive/MyDrive/Datasets/Bac_16s_data/seq_kmers.csv', index = None) #save the kmers file

,Kmers
0,"[GAACGCTGGCGG, AACGCTGGCGGC, ACGCTGGCGGCG, CGC..."
1,"[GTTTGATCCTGG, TTTGATCCTGGC, TTGATCCTGGCT, TGA..."
2,"[TAGAGTTTGATC, AGAGTTTGATCC, GAGTTTGATCCT, AGT..."
3,"[AAGAGTTTGATC, AGAGTTTGATCC, GAGTTTGATCCT, AGT..."
4,"[ATTGAACGCTGG, TTGAACGCTGGC, TGAACGCTGGCG, GAA..."
